In [5]:
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from torchvision.models import resnet50

ImportError: cannot import name 'detect_fake_mode' from 'torch._guards' (C:\Users\jv258\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_guards.py)

In [ ]:
# Set the paths to your training and test data directories
train_data_dir = 'path/to/train/dataset'
test_data_dir = 'path/to/test/dataset'

# Define the image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

In [ ]:
# Preprocess and augment the training data
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
train_dataset = ImageFolder(train_data_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
test_dataset = ImageFolder(test_data_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load the pre-trained ResNet50 model
base_model = resnet50(pretrained=True)

# Freeze the pre-trained layers
for param in base_model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer for the number of classes in your dataset
num_classes = len(train_dataset.classes)
num_features = base_model.fc.in_features
base_model.fc = nn.Linear(num_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = base_model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_predictions / len(train_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

# Save the model
torch.save(model.state_dict(), 'person_recognition_model.pt')

In [ ]:
# Load the pre-trained model
model = resnet50(pretrained=False)
num_classes = len(train_dataset.classes)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model.load_state_dict(torch.load('person_recognition_model.pt'))
model.eval()

# Load Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to perform facial cropping
def crop_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    else:
        return None

In [ ]:
# Function to preprocess the input image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (img_width, img_height))
    image = ToTensor()(image)
    image = image.unsqueeze(0)
    return image

# Load and preprocess the input image
image_path = 'path/to/input/image.jpg'
input_image = cv2.imread(image_path)
cropped_image = crop_face(input_image)

In [ ]:
if cropped_image is not None:
    preprocessed_image = preprocess_image(cropped_image)
    preprocessed_image = preprocessed_image.to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(preprocessed_image)
        _, predicted = torch.max(outputs.data, 1)
        predicted_class = train_dataset.classes[predicted.item()]

        # Display the cropped image and predicted class
        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Predicted Class: {predicted_class}')
        plt.show()
else:
    print('No face detected in the input image.')